# Load data into Cosmos DB using the MongoDB API

This notebook demonstrates how to load data into Cosmos DB from Cosmic Works JSON files into the database using the MongoDB API.

In [1]:
import os
import pymongo
import requests
from pymongo import UpdateOne, DeleteMany
from models import Product, ProductList, Customer, CustomerList, SalesOrder, SalesOrderList
from dotenv import load_dotenv

/Users/shuhaibwalahi/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Establish a connection to the database

In [2]:
load_dotenv()
CONNECTION_STRING = os.environ.get("DB_CONNECTION_STRING")
client = pymongo.MongoClient(CONNECTION_STRING)
# Create database to hold cosmic works data
# MongoDB will create the database if it does not exist
db = client.cosmic_works

In [3]:
# empty the collections
db.products.bulk_write([DeleteMany({})])
db.customers.bulk_write([DeleteMany({})])
db.sales.bulk_write([DeleteMany({})])

BulkWriteResult({'writeErrors': [], 'writeConcernErrors': [], 'nInserted': 0, 'nUpserted': 0, 'nMatched': 0, 'nModified': 0, 'nRemoved': 0, 'upserted': []}, acknowledged=True)

## Load products

In [4]:
# Add product data to database using bulkwrite and updateOne with upsert
# Get cosmic works product data from github
product_raw_data = "https://cosmosdbcosmicworks.blob.core.windows.net/cosmic-works-small/product.json"
product_data = ProductList(items=[Product(**data) for data in requests.get(product_raw_data).json()])
db.products.bulk_write([ UpdateOne({"_id": prod.id}, {"$set": prod.model_dump(by_alias=True)}, upsert=True) for prod in product_data.items])

BulkWriteResult({'writeErrors': [], 'writeConcernErrors': [], 'nInserted': 0, 'nUpserted': 295, 'nMatched': 0, 'nModified': 0, 'nRemoved': 0, 'upserted': [{'index': 0, '_id': '027D0B9A-F9D9-4C96-8213-C8546C4AAE71'}, {'index': 1, '_id': '08225A9E-F2B3-4FA3-AB08-8C70ADD6C3C2'}, {'index': 2, '_id': '0A7E57DA-C73F-467F-954F-17B7AFD6227E'}, {'index': 3, '_id': '14174164-F6C0-47FC-83FB-604C6A63408D'}, {'index': 4, '_id': '1A176FDB-D9A8-4888-BDD9-CE4F12E97AAE'}, {'index': 5, '_id': '201D0D79-81AD-43D2-AD6E-F09EEE6AC2D7'}, {'index': 6, '_id': '24BE4267-85D8-4C1A-B184-C08709495752'}, {'index': 7, '_id': '290B4594-95BE-47C5-863A-4EFAAFC0AED7'}, {'index': 8, '_id': '29663491-D2E9-47B4-83AE-D9459B6B5B67'}, {'index': 9, '_id': '2C981511-AC73-4A65-9DA3-A0577E386394'}, {'index': 10, '_id': '3F105575-8677-42F9-8E1F-76E4B450F136'}, {'index': 11, '_id': '3FE1A99E-DE14-4D11-B635-F5D39258A0B9'}, {'index': 12, '_id': '44873725-7B3B-4B28-804D-963D2D62E761'}, {'index': 13, '_id': '47C70E1E-E500-41B3-8615-DCC

## Load customers and sales raw data

In this repository, the customer and sales data are stored in the same file. The `type` field is used to differentiate between the two types of documents.

In [5]:
customer_sales_raw_data = "https://cosmosdbcosmicworks.blob.core.windows.net/cosmic-works-small/customer.json"
response = requests.get(customer_sales_raw_data)
# override decoding
response.encoding = 'utf-8-sig'
response_json = response.json()
# filter where type is customer
customers = [cust for cust in response_json if cust["type"] == "customer"]
# filter where type is salesOrder
sales_orders = [sales for sales in response_json if sales["type"] == "salesOrder"]

## Load customers

In [6]:
customer_data = CustomerList(items=[Customer(**data) for data in customers])
db.customers.bulk_write([ UpdateOne({"_id": cust.id}, {"$set": cust.model_dump(by_alias=True)}, upsert=True) for cust in customer_data.items])

BulkWriteResult({'writeErrors': [], 'writeConcernErrors': [], 'nInserted': 0, 'nUpserted': 100, 'nMatched': 0, 'nModified': 0, 'nRemoved': 0, 'upserted': [{'index': 0, '_id': '022BB1FA-35E6-4CC5-9079-8EA61FE7FAAE'}, {'index': 1, '_id': '0E57A241-1B95-43A2-BCFB-637608B0AD1A'}, {'index': 2, '_id': '23A65A9A-479C-44D2-9F6A-E6CDA8B0BE08'}, {'index': 3, '_id': '29C95F8A-9C52-48DB-A1C4-8A14C430FF06'}, {'index': 4, '_id': '34E7A125-0F66-4673-A80B-20B4C46EAD3A'}, {'index': 5, '_id': '35D52474-3D1A-433C-A310-10FA7DF8950B'}, {'index': 6, '_id': '3945DB3E-2632-466C-BCBE-0C252729C937'}, {'index': 7, '_id': '44A6D5F6-AF44-4B34-8AB5-21C5DC50926E'}, {'index': 8, '_id': '45E422FD-0AE2-4C73-8883-61B1C3BB4431'}, {'index': 9, '_id': '4FEAA310-61D4-4A89-8E78-3CA6B34F7934'}, {'index': 10, '_id': '537E369C-C65B-4F23-B7C0-D07DFFFAC08B'}, {'index': 11, '_id': '5EE9C404-EBE5-45F6-8063-E537AA5E750C'}, {'index': 12, '_id': '6325847B-D85C-4F23-9C47-7346082D38A1'}, {'index': 13, '_id': '670C1D45-DCAF-4B08-8358-F1D

## Load sales orders

In [7]:
sales_data = SalesOrderList(items=[SalesOrder(**data) for data in sales_orders])
db.sales.bulk_write([ UpdateOne({"_id": sale.id}, {"$set": sale.model_dump(by_alias=True)}, upsert=True) for sale in sales_data.items])

BulkWriteResult({'writeErrors': [], 'writeConcernErrors': [], 'nInserted': 0, 'nUpserted': 1484, 'nMatched': 0, 'nModified': 0, 'nRemoved': 0, 'upserted': [{'index': 0, '_id': '00500AA1-3E9D-4E83-9C21-07B0AF482B3F'}, {'index': 1, '_id': '00CC8882-4B98-4273-BDD3-732CB8F5A2E0'}, {'index': 2, '_id': '00E4B453-A79A-4590-8CCB-28ED95003CB7'}, {'index': 3, '_id': '0148B088-5124-45FB-B815-22851683D8AA'}, {'index': 4, '_id': '01AFA50A-2009-4AC3-A008-B6F8E2003BB6'}, {'index': 5, '_id': '01D792C9-8C84-4120-B9FB-87C707D94B30'}, {'index': 6, '_id': '0288E4FA-1C89-411E-801E-5098B0CDC414'}, {'index': 7, '_id': '029E366C-0023-40AB-9684-51F8BC734BCB'}, {'index': 8, '_id': '0379883C-C865-4D57-BF65-AF79267CE961'}, {'index': 9, '_id': '039896DA-698A-4C60-8A6B-083926A5C281'}, {'index': 10, '_id': '04B0E36F-E776-4B93-89A2-3FDD59B4F0B5'}, {'index': 11, '_id': '05223D68-4D04-4EF3-A4E2-BE7DEA6A8066'}, {'index': 12, '_id': '05F826D9-3526-463D-96CE-A2340DE8E554'}, {'index': 13, '_id': '06794E40-1A3E-49B6-9914-94

## Clean up

In [8]:
client.close()